In [ ]:
import cv2
import numpy as np
import os
import csv
from Models import FullModel
from GazeEyeChimera import Detector, Predictor
from utils import (
    get_config,
)

SETTINGS, COLOURS, EYETRACKER, TF = get_config("config.ini")

detector = Detector(output_size=SETTINGS["image_size"])
predictor = Predictor(
    FullModel,
    model_data="trained_models/eyetracking_model.pt",
    config_file="trained_models/eyetracking_config.json",
)
screen_errors = region_map = np.load("trained_models/eyetracking_errors.npy")

# Setup directories to save
data_dirs = (
    "data/EyeChimera/l_eye",
    "data/EyeChimera/r_eye",
    "data/EyeChimera/face",
    "data/EyeChimera/face_aligned",
    "data/EyeChimera/head_pos",
)

# Setup directories to upload

diretorios = [
    "Bases/EyeChimeraData/00.Centre",
    "Bases/EyeChimeraData/01.UpRight",
    "Bases/EyeChimeraData/02.UpLeft",
    "Bases/EyeChimeraData/03.Right",
    "Bases/EyeChimeraData/04.Left",
    "Bases/EyeChimeraData/05.DownRight",
    "Bases/EyeChimeraData/06.DownLeft"
]

for d in data_dirs:
    if not os.path.exists(d):
        os.makedirs(d)

data_file_path = "data/EyeChimera/positions.csv"
data_file_exists = os.path.isfile(data_file_path)
data_file = open(data_file_path, "a", newline="")
csv_writer = csv.writer(data_file, delimiter=",")

if not data_file_exists:
    csv_writer.writerow(["id", "file", "class", "detector", "x", "y", "head_angle"])

num_images = len(os.listdir("data/EyeChimera/face"))

def save_data(
    num_images,
    l_eye,
    r_eye,
    face,
    face_align,
    head_pos,
    angle,
    targetx,
    targety,
    DetectorCapture,
    file,
    nome_ultimo_diretorio,
):
    data_id = num_images + 1

    for (path, img) in zip(data_dirs, (l_eye, r_eye, face, face_align, head_pos)):
        cv2.imwrite("{}/{}.jpg".format(path, data_id), img)

    csv_writer.writerow([data_id, file, nome_ultimo_diretorio, DetectorCapture, targetx, targety, angle])

    return data_id


for diretorio in diretorios:
    nome_ultimo_diretorio = os.path.basename(diretorio)
    for root, dirs, files in os.walk(diretorio):
        for file in files:
            if '.jpg' in file:            
                img = cv2.imread("Bases/EyeChimeraData/"  + str(nome_ultimo_diretorio) + "/" + str(file), cv2.IMREAD_COLOR)
                print (nome_ultimo_diretorio)
                print (file)
                print (img)
                m = np.ones((3,3))
                #frame = cv2.transform(img, m, None)
                img = cv2.transform(img, m, None)
                DetectorCapture, l_eye, r_eye, face, face_align, head_pos, angle = detector.get_frame(img)
                
                if DetectorCapture == True:
                    x_hat, y_hat = predictor.predict(
                        face, l_eye, r_eye, head_pos, head_angle=angle
                        )
                else:
                    x_hat, y_hat, angle = None, None, None
                    
                num_images = save_data(
                    num_images,
                    l_eye,
                    r_eye,
                    face,
                    face_align,
                    head_pos,
                    angle,
                    x_hat,
                    y_hat,
                    DetectorCapture,
                    file,
                    nome_ultimo_diretorio,
                )